In [1]:
import json
import subprocess
from pathlib import Path
from tempfile import NamedTemporaryFile
from typing import Literal

In [2]:
%%sh

CPU=$(($(nproc) - 1))

for i in `seq 0 $CPU`
do
  sudo cpufreq-set -c $i -g userspace
  sudo cpufreq-set -c $i -d 2.90GHz
  sudo cpufreq-set -c $i -u 2.90GHz
  sudo cpufreq-set -c $i -f 2.90GHz
done

In [9]:
JACCARD_BINARY = Path("../build/jaccard")


def run_jaccard_evolve(
    graph: Literal["LS_CSR", "CSR_64", "MOR_GR", "LC_MOR_GR"] = "LS_CSR",
    n: int = 10_000,
    m: int = 10,
    p: float = 0.15,
):
    with NamedTemporaryFile(
        "r", prefix=f"{graph}-n={n}-m={m}-p={p:.2f}-", suffix=".txt", delete=False
    ) as stats_file:
        jaccard_proc = subprocess.run(
            [
                JACCARD_BINARY,
                f"-g={graph}",
                f"-n={n}",
                f"-m={m}",
                f"-p={p}",
                "--style=EVOLVE",
                "--out=NON",
                "--stats",
                stats_file.name,
            ],
        )
        jaccard_proc.check_returncode()

        results = list(map(json.loads, stats_file.readlines()))
        for result in results:
            (g, _e, stage, step) = result["name"].split()
            (step, max_step) = tuple(map(int, step.split("/")))
            result["graph"] = g
            result["stage"] = stage
            result["step"] = step
            result["max_step"] = max_step

        return results

In [10]:
results = []

In [12]:
exp_args = {"n": 100, "m": 10, "p": 0.15}
for graph in ["LS_CSR", "LC_MOR_GR"]:
    results += run_jaccard_evolve(graph=graph, **exp_args)

File Read Time (ns):	742431
STAT_TYPE, REGION, CATEGORY, TOTAL_TYPE, TOTAL
STAT, Sorting 1, Iterations, TSUM, 42
STAT, Sorting 1, Time, TMAX, 2
STAT, Insertion 1, Iterations, TSUM, 200
STAT, Sorting 0, Iterations, TSUM, 1932
STAT, Insertion 0, Iterations, TSUM, 9200
STAT, Sorting 2, Iterations, TSUM, 42
STAT, Insertion 2, Iterations, TSUM, 200
STAT, Sorting 3, Iterations, TSUM, 42
STAT, Insertion 3, Iterations, TSUM, 200
STAT, Sorting 4, Iterations, TSUM, 42
STAT, Insertion 4, Iterations, TSUM, 200
STAT, Sorting 5, Iterations, TSUM, 42
STAT, Insertion 5, Iterations, TSUM, 200
STAT, Sorting 6, Iterations, TSUM, 42
STAT, Insertion 6, Iterations, TSUM, 200
STAT, Sorting 7, Iterations, TSUM, 42
STAT, Insertion 7, Iterations, TSUM, 200
STAT, Sorting 8, Iterations, TSUM, 42
STAT, Insertion 8, Iterations, TSUM, 200


jaccard: for the -g option: Cannot find option named 'LC_MOR_GR'!


CalledProcessError: Command '[PosixPath('../build/jaccard'), '-g=LC_MOR_GR', '-n=100', '-m=10', '-p=0.15', '--style=EVOLVE', '--out=NON', '--stats', '/tmp/LC_MOR_GR-n=100-m=10-p=0.15-l6mfqc92.txt']' returned non-zero exit status 1.

In [ ]:
# todo: for each stage, plot graph of cycles vs step, with cycles grouped by graph.

In [5]:
%%sh

CPU=$(($(nproc) - 1))

for i in `seq 0 $CPU`
do
  sudo cpufreq-set -c $i -g powersave
  sudo cpufreq-set -c $i -u 3.30GHz
  sudo cpufreq-set -c $i -d 1.20GHz
done